<a href="https://colab.research.google.com/github/jwk515/FantasyQuickDash_DEV/blob/main/Fantasy_Quick_Dash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Fantasy Quick Dash
## Team: Krisna Patel, Sam Wexler, Jack Kerin

We were able to integrate Yahoo’s Fantasy API into the format that we want our output to be, however, we later realized that the Football season scoring isn’t being tracked for Pre-Season fantasy by Yahoo Fantasy. Thus, we need to get Fantasy information for the MLB or another sport that is currently running. We have signed up for an NFL draft on August 8th to see if that information will port over. Considering this, we may need to unfortunately pivot from Yahoo Fantasy to another API as Yahoo Fantasy has blocked registration for the 2025 season of MLB or we may adjust the focus of this project from being a fantasy team tracker, due to the above reasons, to being a draft predictor for the NFL draft. Following the creation of the code in CoLab, we will look to transition this to local development / HTML.

In [ ]:
# Log in with your Yahoo account, pick a sport and week, and see that week’s head-to-head scoreboard in one click.
# I want a dashboard that automatically lists my leagues so I only have to choose the week to check my matchup score.
# ChatGPT O3 was utilized to understand the coding for token login support

!pip -q install requests

import os, json, requests
from google.colab import userdata

# ---------- 1. keys -----------------------------------------------
CLIENT_ID     = userdata.get("CLIENT_ID")
CLIENT_SECRET = userdata.get("CLIENT_SECRET")
if not CLIENT_ID or not CLIENT_SECRET:
    CLIENT_ID     = input("Yahoo CLIENT_ID : ").strip()
    CLIENT_SECRET = input("Yahoo CLIENT_SECRET : ").strip()

REDIRECT_URI = "oob"
SCOPE        = "fspt-r"
TOKEN_FILE   = "yahoo_token.json"

# ---------- 2. login helpers --------------------------------------
def save_token(t):
    json.dump(t, open(TOKEN_FILE, "w"))

def get_new_tokens():
    url = ( "https://api.login.yahoo.com/oauth2/request_auth?"
            f"client_id={CLIENT_ID}&redirect_uri={REDIRECT_URI}"
            f"&response_type=code&scope={SCOPE}" )
    print("\nSTEP 1 – open, sign-in, click Allow:\n", url, "\n")
    full = input("STEP 2 – paste the ENTIRE URL afterwards: ").strip()
    if "code=" not in full:
        raise ValueError("No ?code= found.")
    code = full.split("code=")[1].split("&")[0]

    t = requests.post(
        "https://api.login.yahoo.com/oauth2/get_token",
        auth=(CLIENT_ID, CLIENT_SECRET),
        data={"grant_type":"authorization_code",
              "redirect_uri":REDIRECT_URI,
              "code":code},
        timeout=30).json()
    if "access_token" not in t:
        raise SystemExit("Yahoo replied:\n"+json.dumps(t,indent=2))
    save_token(t)
    return t

def refresh_tokens(r_token):
    t = requests.post(
        "https://api.login.yahoo.com/oauth2/get_token",
        auth=(CLIENT_ID, CLIENT_SECRET),
        data={"grant_type":"refresh_token",
              "redirect_uri":REDIRECT_URI,
              "refresh_token":r_token},
        timeout=30).json()
    if "access_token" in t:
        print("✓ Access token refreshed.\n")
        save_token(t)
        return t
    else:
        print("Could not refresh:\n", json.dumps(t, indent=2))
        os.remove(TOKEN_FILE)
        return get_new_tokens()

# ---------- 3. load or obtain token -------------------------------
if os.path.exists(TOKEN_FILE):
    token = json.load(open(TOKEN_FILE))
    if "access_token" not in token:
        token = get_new_tokens()
else:
    token = get_new_tokens()

ACCESS_TOKEN   = token["access_token"]
REFRESH_TOKEN  = token.get("refresh_token")

# ---------- 4. Yahoo GET helper with auto-refresh -----------------
def yget(path):
    """GET <path>?format=json  – auto-refreshes once on 401/token_expired"""
    global ACCESS_TOKEN, token
    url = f"https://fantasysports.yahooapis.com/fantasy/v2/{path}?format=json"
    hdr = {"Authorization": f"Bearer {ACCESS_TOKEN}"}
    r   = requests.get(url, headers=hdr, timeout=30)

    if r.status_code == 401 or "token_expired" in r.text:
        if not REFRESH_TOKEN:
            raise SystemExit("Token expired and no refresh_token saved – run login again.")
        token = refresh_tokens(REFRESH_TOKEN)
        ACCESS_TOKEN = token["access_token"]
        hdr["Authorization"] = f"Bearer {ACCESS_TOKEN}"
        r = requests.get(url, headers=hdr, timeout=30)   # retry once
    r.raise_for_status()
    return r.json()

print("ready — token valid for this session\n")

# ---------- 5. choose league / week -------------------------------
sport  = input("Sport (nfl / mlb / nba / nhl): ").strip().lower()
league = input("League ID (digits only)        : ").strip()
week   = input("Week number                    : ").strip()
league_key = f"{sport}.l.{league}"

# ---------- 6. weekly scoreboard ----------------------------------
raw = yget(f"league/{league_key}/scoreboard;week={week}")
if "fantasy_content" not in raw:
    raise SystemExit("Yahoo returned:\n"+json.dumps(raw,indent=2))

scoreboard = raw["fantasy_content"]["league"][1]["scoreboard"]["0"]
matchups   = scoreboard.get("matchups", {})

print(f"\n=== {sport.upper()} — Week {week} Scores ===")
if "count" not in matchups:
    print("(No match-ups found — maybe the week hasn’t started yet.)")
else:
    for m in matchups.values():
        if isinstance(m, dict):
            teams_block = m["matchup"].get("teams", {})
            for blob in teams_block.values():
                if isinstance(blob, dict):
                    team = blob["team"]
                    name = team[0]["name"]
                    pts  = team[1]["team_points"]["total"]
                    print(f"{name:<30} {pts} pts")

# ---------- 7. league standings -----------------------------------
st = yget(f"league/{league_key}/standings")

stand_block = st["fantasy_content"]["league"][1]["standings"]
teams_obj   = stand_block[0]["teams"] if isinstance(stand_block, list) \
             else stand_block["0"]["teams"]

print("\n=== Current Standings (W-L-T) ===")
for raw_team in (teams_obj.values() if isinstance(teams_obj, dict) else teams_obj):

    # flat list ----------------
    if isinstance(raw_team, dict) and "team" in raw_team:
        parts = raw_team["team"]
    elif isinstance(raw_team, list):
        parts = raw_team
    else:
        continue

    # find the team name ---------------------
    name_dict = next((p for p in parts if isinstance(p, dict) and "name" in p),
                     None)
    if not name_dict:
        continue
    name = name_dict["name"]

    # find the team_standings -----------------------
    stand_dict = next((p for p in parts if isinstance(p, dict)
                       and "team_standings" in p), None)
    if not stand_dict:
        print(f"{name:<30}  (no standings data)")
        continue

    rec = stand_dict["team_standings"]["outcome_totals"]
    print(f"{name:<30}  {rec['wins']}-{rec['losses']}-{rec['ties']}")

# ---------- 8. rosters for the week -------------------------------
print(f"\n=== Rosters — Week {week} ===")

unique_team_keys = set()

for match in matchups.values():
    if not isinstance(match, dict):
        continue

    mcup = match.get("matchup", match)
    teams_obj = mcup.get("teams")
    if teams_obj is None:
        continue

    team_entries = (teams_obj.values() if isinstance(teams_obj, dict)
                    else teams_obj)

    for blob in team_entries:
        if isinstance(blob, dict) and "team" in blob:
            unique_team_keys.add(blob["team"][0]["team_key"])

if not unique_team_keys:
    print("(No rosters — this week has only byes or the season hasn’t started.)")

for tkey in unique_team_keys:
    roster_json = yget(f"team/{tkey}/roster;week={week}")
    team_name   = roster_json["fantasy_content"]["team"][0]["name"]
    players_obj = roster_json["fantasy_content"]["team"][1]["roster"]["0"]["players"]

    print(f"\n{team_name}")
    for ply in players_obj.values():
        if isinstance(ply, dict):
            print("  •", ply["player"][0]["name"]["full"])


STEP 1 – open, sign-in, click Allow:
 https://api.login.yahoo.com/oauth2/request_auth?client_id=dj0yJmk9c3VPQUh5SlVhNWk1JmQ9WVdrOU9IQlFiRUl6VFVjbWNHbzlNQT09JnM9Y29uc3VtZXJzZWNyZXQmc3Y9MCZ4PTc3&redirect_uri=oob&response_type=code&scope=fspt-r 

STEP 2 – paste the ENTIRE URL afterwards: https://api.login.yahoo.com/oauth2/code?client_id=dj0yJmk9c3VPQUh5SlVhNWk1JmQ9WVdrOU9IQlFiRUl6VFVjbWNHbzlNQT09JnM9Y29uc3VtZXJzZWNyZXQmc3Y9MCZ4PTc3&code=xpkhnjm&crumb=BTMvF2ZISio&signature=1%7ELXBwzo67kSO-XAo7NdtHd5BDixvTzfHFHKvCmcZ4IP4&src=oauth
ready — token valid for this session

Sport (nfl / mlb / nba / nhl): nfl
League ID (digits only)        : 1111145
Week number                    : 1

=== NFL — Week 1 Scores ===

=== Current Standings (W-L-T) ===

=== Rosters — Week 1 ===
(No rosters — this week has only byes or the season hasn’t started.)


In [ ]:
# ── INSTALL & IMPORT ─────────────────────────────────────────────────────────
!pip install requests tabulate

import os
import requests
from tabulate import tabulate

# ── CONFIG ────────────────────────────────────────────────────────────────────
API_KEY = os.getenv("X_RAPIDAPI_KEY") or input("Paste your RapidAPI Key: ").strip()
BASE_URL = "https://tank01-nfl-live-in-game-real-time-statistics-nfl.p.rapidapi.com"
HEADERS = {
    "x-rapidapi-host": BASE_URL.replace("https://",""),
    "x-rapidapi-key":  API_KEY
}

def api_get(endpoint, params=None):
    resp = requests.get(f"{BASE_URL}/{endpoint}", headers=HEADERS, params=params or {})
    resp.raise_for_status()
    return resp.json()

# ── 1. USER INPUT ─────────────────────────────────────────────────────────────
n_teams = int(input("How many teams? ").strip())
slot    = int(input(f"Your draft slot (1–{n_teams})? ").strip())
season  = input("Season year (e.g. 2025): ").strip()
print(f"\nLeague size: {n_teams} • Your slot: {slot} • Season: {season}")

adp_fmt = input("ADP format [standard,PPR,halfPPR,bestBall,IDP,superFlex]: ").strip() or "standard"

# ── 2. FETCH ADP & BUILD BUCKETS ───────────────────────────────────────────────
print("Fetching ADP…", end="", flush=True)
adp_data = api_get("getNFLADP", {"season": season, "adpType": adp_fmt})
adp_list = adp_data.get("body",{}).get("adpList", [])
print(" ✓ ADP loaded." if adp_list else " ⚠️ no ADP data.")

positions = ["QB","RB","WR","TE","K","DST"]
buckets   = {pos: [] for pos in positions}

for p in adp_list:
    raw = p.get("posADP","").upper()
    if raw.startswith("DST") or raw=="DEF":
        pos = "DST"
    elif raw.startswith("K"):
        pos = "K"
    else:
        pos = raw[:2]

    if pos in buckets:
        buckets[pos].append({
            "name":     p.get("longName","<none>"),
            "adp":      float(p.get("overallADP",0)),
            "playerID": p.get("playerID"),
            "pos":      pos
        })

# ── 3. DST / K FALLBACK ───────────────────────────────────────────────────────
if not buckets["DST"]:
    print("No DST in ADP → adding DST fallback…", end="", flush=True)
    teams_resp = api_get("getNFLTeams", {"teamStats":"true"})
    teams = teams_resp.get("body",{}).get("teams",[]) if isinstance(teams_resp,dict) else teams_resp
    for t in teams[:n_teams]:
        abv = t.get("teamAbv","UNK")
        buckets["DST"].append({"name":f"{abv} DST","adp":200.0,"playerID":None,"pos":"DST"})
    print(" ✓ done.")

if not buckets["K"]:
    print("No K in ADP → adding K fallback…", end="", flush=True)
    for _ in range(n_teams):
        buckets["K"].append({"name":"<none K>","adp":999.0,"playerID":None,"pos":"K"})
    print(" ✓ done.")

# ── 4. DRAFT SETUP ─────────────────────────────────────────────────────────────
starting_limits = {"QB":1,"RB":2,"WR":3,"TE":1,"K":1,"DST":1}
ROSTER_SIZE     = 15
BENCH_SLOTS     = ROSTER_SIZE - sum(starting_limits.values())

rosters = {i: [] for i in range(1, n_teams+1)}
draft_log = []
total_picks = n_teams * ROSTER_SIZE

# ── 5. SNAKE DRAFT ─────────────────────────────────────────────────────────────
for pick in range(total_picks):
    rnd    = pick // n_teams + 1
    idx    = pick % n_teams + 1
    team   = idx if rnd%2==1 else (n_teams+1-idx)

    # see which starters this team still needs
    have = {pos: sum(1 for pl in rosters[team] if pl["pos"]==pos)
            for pos in starting_limits}
    needed = [pos for pos,cap in starting_limits.items() if have[pos]<cap]

    if needed:
        # pick the lowest‐ADP player among needed positions
        pool = [buckets[pos][0] for pos in needed if buckets[pos]]
        sel  = min(pool, key=lambda x: x["adp"])
    else:
        # bench pick: best ADP among *all* remaining
        flat = [pl for pos in buckets for pl in buckets[pos]]
        sel  = min(flat, key=lambda x: x["adp"]) if flat else {"name":"<none>","adp":999,"playerID":None,"pos":"RB"}

    # remove from that bucket
    if sel["pos"] in buckets and sel in buckets[sel["pos"]]:
        buckets[sel["pos"]].remove(sel)

    rosters[team].append(sel)
    draft_log.append([rnd, pick+1, team, sel["pos"], sel["name"]])

# ── 6. PRINT FULL MOCK DRAFT ─────────────────────────────────────────────────
print("\n=== Full Mock Draft ===")
print(tabulate(draft_log,
               headers=["Rnd","Pick#","Team","Pos","Player"],
               tablefmt="github"))

# ── 7. SHOW YOUR 15-MAN ROSTER ────────────────────────────────────────────────
def show_roster(num):
    print(f"\n=== Team {num} (Full {ROSTER_SIZE}-man Roster) ===")
    tbl = [[i+1, pl["pos"], pl["name"], f"{pl['adp']:.1f}"]
           for i,pl in enumerate(rosters[num])]
    print(tabulate(tbl, headers=["#","Pos","Name","ADP"], tablefmt="github"))

show_roster(slot)
peek = input(f"\nPeek another team? (1–{n_teams} or ENTER to skip): ").strip()
if peek.isdigit(): show_roster(int(peek))

# ── 8. WEEKLY PROJECTIONS & STARTING LINEUP ────────────────────────────────────
wk = int(input("\nWhich week (1–18)? ").strip())
print(f"\nFetching projections for Week {wk}…")

proj_payload = {
    "week":               wk,
    "archiveSeason":      season,
    "twoPointConversions":2, "passYards": 0.04, "passTD":    4,
    "passCompletions":    1, "passInterceptions":-2,
    "pointsPerReception": 1, "rushYards":   0.1, "rushTD":    6,
    "receivingYards":     0.1, "receivingTD":    6, "fumbles": -2,
    "fgMade":             3, "fgMissed":      -1,
    "xpMade":             1, "xpMissed":     -1
}

resp = api_get("getNFLProjections", proj_payload)
body = resp.get("body",{}) if isinstance(resp,dict) else resp
all_proj = body.get("playerProjections", {})

proj_tbl  = []
total_pts = 0.0

for pl in rosters[slot]:
    pid = pl.get("playerID")
    rec = all_proj.get(str(pid), {}) if pid else {}
    # first try the live field
    fp  = rec.get("fantasyPoints")
    # fall back to default if missing
    if fp is None:
        fp = rec.get("fantasyPointsDefault", {}).get(adp_fmt)
    try:
        pts = float(fp or 0)
    except:
        pts = 0.0
    proj_tbl.append([pl["name"], pl["pos"], f"{pts:.1f}"])
    total_pts += pts

# grand total row
proj_tbl.append(["**Total**", "", f"**{total_pts:.1f}**"])
print("\n" + tabulate(proj_tbl,
                     headers=["Player","Pos","ProjPts"],
                     tablefmt="github"))

# PROPOSED STARTING LINEUP
print(f"\n=== Proposed Starting Lineup for Team {slot} ===")
name_to_pts = {r[0]: float(r[2]) for r in proj_tbl if r[0]!="**Total**"}
lineup = []
for pos,cap in starting_limits.items():
    candidates = sorted(
        [(pl["name"], name_to_pts.get(pl["name"],0.0))
         for pl in rosters[slot] if pl["pos"]==pos],
        key=lambda x: x[1], reverse=True
    )
    for name,pts in candidates[:cap]:
        lineup.append([pos,name,f"{pts:.1f}"])

print(tabulate(lineup, headers=["Pos","Name","ProjPts"], tablefmt="github"))
print("\n✅ Done!")


Paste your RapidAPI Key: d0e597ad18msh5be29b471d220f2p118766jsnfc20e02bcf2c
How many teams? 10
Your draft slot (1–10)? 2
Season year (e.g. 2025): 2025

League size: 10 • Your slot: 2 • Season: 2025
ADP format [standard,PPR,halfPPR,bestBall,IDP,superFlex]: standard
Fetching ADP… ✓ ADP loaded.

=== Full Mock Draft ===
|   Rnd |   Pick# |   Team | Pos   | Player                   |
|-------|---------|--------|-------|--------------------------|
|     1 |       1 |      1 | WR    | Ja'Marr Chase            |
|     1 |       2 |      2 | RB    | Saquon Barkley           |
|     1 |       3 |      3 | RB    | Bijan Robinson           |
|     1 |       4 |      4 | WR    | Justin Jefferson         |
|     1 |       5 |      5 | RB    | Jahmyr Gibbs             |
|     1 |       6 |      6 | WR    | CeeDee Lamb              |
|     1 |       7 |      7 | RB    | Derrick Henry            |
|     1 |       8 |      8 | WR    | Nico Collins             |
|     1 |       9 |      9 | WR    | Puka 